In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext

sc = SparkContext("local", "first app")

In [8]:
data_A = sc.parallelize([("A", 0, 0, 1), ("A", 0, 1, 2), ("A", 1, 0, 3), ("A", 1, 1, 4)])
data_B = sc.parallelize([("B", 0, 0, 5), ("B", 0, 1, 6), ("B", 1, 0, 7), ("B", 1, 1, 8)])

In [9]:
print(data_A.collect())

[('A', 0, 0, 1), ('A', 0, 1, 2), ('A', 1, 0, 3), ('A', 1, 1, 4)]


In [58]:
data_A_map = data_A.flatMap(lambda x : [((x[1], i), (x[2], x[3])) for i in range(2)])

In [59]:
print(data_A_map.collect())

[((0, 0), (0, 1)), ((0, 1), (0, 1)), ((0, 0), (1, 2)), ((0, 1), (1, 2)), ((1, 0), (0, 3)), ((1, 1), (0, 3)), ((1, 0), (1, 4)), ((1, 1), (1, 4))]


In [60]:
data_B_map = data_B.flatMap(lambda x : [((i, x[2]), (x[1], x[3])) for i in range(2)])

In [61]:
print(data_B_map.collect())

[((0, 0), (0, 5)), ((1, 0), (0, 5)), ((0, 1), (0, 6)), ((1, 1), (0, 6)), ((0, 0), (1, 7)), ((1, 0), (1, 7)), ((0, 1), (1, 8)), ((1, 1), (1, 8))]


In [62]:
data_multiple = data_A_map.union(data_B_map)

In [63]:
print(data_multiple.collect())

[((0, 0), (0, 1)), ((0, 1), (0, 1)), ((0, 0), (1, 2)), ((0, 1), (1, 2)), ((1, 0), (0, 3)), ((1, 1), (0, 3)), ((1, 0), (1, 4)), ((1, 1), (1, 4)), ((0, 0), (0, 5)), ((1, 0), (0, 5)), ((0, 1), (0, 6)), ((1, 1), (0, 6)), ((0, 0), (1, 7)), ((1, 0), (1, 7)), ((0, 1), (1, 8)), ((1, 1), (1, 8))]


In [76]:
def matmul(values) : 
    tmp = [1 for _ in range(2)]
    for (k, v) in values : 
        tmp[k] = tmp[k] * v
    return sum(tmp)

In [77]:
data_C = data_multiple.groupByKey().mapValues(matmul)

In [78]:
print(data_C.collect())

[((0, 0), 19), ((1, 1), 50), ((0, 1), 22), ((1, 0), 43)]


In [91]:
data_C_sum = data_C.map(lambda x : x[1])

In [94]:
print(data_C_sum.reduce(lambda x, y : x + y))

134
